In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Cargamos el modelo de reconocimiento de entidades nombradas (NER) en español de Flair
## Fuente: https://huggingface.co/flair/ner-spanish-large

Flair es un marco de trabajo para procesamiento de lenguaje natural (NLP) que permite aplicar modelos de vanguardia a su texto para la posterior clasificación que será exportada a formato .JSON


In [ ]:
#importe los módulos necesarios.
import requests
import regex as re
import pandas as pd
import json
import flair
import numpy as np
# Luego, importe las clases o funciones específicas que se utilizarán
from bs4 import BeautifulSoup
from flair.data import Sentence
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from flair.models import SequenceTagger


El modelo "flair/ner-spanish-large" predice 4 etiquetas: PER (nombre de persona), LOC (nombre de lugar), ORG (nombre de organización) y MISC (otro nombre).



In [ ]:
# Cargamos el modelo ner-spanish-large de Flair
TAGGER = SequenceTagger.load("flair/ner-spanish-large")
# El modelo está basado en embeddings de nivel documental con XLM-R y FLERT

2023-06-11 17:12:09,882 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


Esta función lee un archivo de Excel y lo convierte en un DataFrame de pandas. También elimina las filas que tienen la columna 'TEXTO' vacía, ya que no son útiles para el análisis.

In [ ]:
def load_data(path_to_filename: str) -> pd.DataFrame:
    df = pd.read_excel(path_to_filename) # Leemos el archivo de Excel con pandas
    df.dropna(subset=['TEXTO'], inplace=True) # Descartamos las filas con valores nulos en 'TEXTO'
    return df # Retornamos el DataFrame limpio

Esta función toma una entidad etiquetada por el modelo de Flair y la transforma en una tupla con el texto de la entidad, el tipo de entidad y la probabilidad con la que se detecta la entidad. Así, se crea una función llamada get_impact(text) que recibe un texto como argumento y retorna la categoría de impacto, sin embargo, la precisión en este proceso puede mejorarse en desarrollos posteriores.



In [ ]:
def entity_from_tagged_str(entity: flair.data.Span) -> tuple:
    parsed_entity = re.findall('"([^"]*)"', str(entity))
    confidence = re.findall('[^()]+(?=\))', str(entity))
    clss = ''
    if 'PER' in str(entity):
        clss = 'PER'
    elif 'LOC' in str(entity):
        clss = 'LOC'
    elif 'ORG' in str(entity):
        clss = 'ORG'
    elif 'MISC' in str(entity):
        clss = 'MISC'
    return parsed_entity[0], clss, confidence[0]

Entrenamiento con textos a elección

In [ ]:
def get_impact(text: str) -> str:
    # 1. Entrenamiento con datos y clasificacion dada en los datos
    textos = [
        'Contaminacion', 'La afectación ambiental ocasionada por el derrame de cerca de 130.000 galones de crudo obligado por las Farc en putumayo se prolonga por meses, e incluso años, en la vegetación y las fuentes hídricas.', 'Afectación por contaminacion con hidrocarburos en afluentes hidricos en Puerto Asis que presta el servicio a 9.000 habitantes del municipio. Sobre esta situación se busca la identificación y evaluación para determinar el grado de contaminación de la Quebrada Agua Negra que surte agua para el cinsumo del municipio.', 'En 2019 se han registrado 19 atentados contra este oleoducto que transporta el crudo entre Putumayo y Nariño hacia el puerto de Tumaco, en el Pacífico. El derrame de crudo afecto el rio Guamuez en Orito Putumayo.', 'el derrame del hidrocarburo afectó 9 humedales; 7 en la vereda Las Cabañas y 2 más en la Vereda de Santa Rosa, lo que suma 3,5 hectáreas de humedales afectados, causando un grave deterioro al ecosistema de fauna y flora en la región.', 'En las localidades de Curillo, Solita y Solano fue declarada la emergencia sanitaria para prevenir la captación de agua para sus acueductos. Empresa petrolera activó plan de contingencia. Se cree que contaminación fue producida por una extracción ilegal de crudo.', 'Comunidades que habitan en la Inspección de Puerto Limón evidenciaron los olores a petróleo en el momento que crecía el río Mocoa por las fuertes lluvias en la zona. Se trató del daño en la tubería que atraviesa el río y conduce gas y crudo lo que originó la contaminación hacia aguas del río Caquetá.', 'autoridades evalúan daños ambientales generados por el derrame de crudo; a causa del ataque perpetrado contra dos carrotanques en la vía que de Puerto Rico conduce hacia San Vicente del Caguán en el norte del Caquetá.', 'Una embarcación naufragó en el río con 12.000 galones de gasolina y 8.000 de Acpm. El barco “Luna Azul” de la empresa Transportes Caribe, que cubría la ruta Puerto Asís- Puerto Leguízamo, se volteó a 20 kilómetros del destino.', 'La contaminación que provoca este metal es principalmente dañina para la principal fuente de proteína del territorio, si se ingiere sin control, causa daños a la salud como dañar sus riñones y el sistema cardiovascular  porque está contaminada.', 'El Guainía es sinónimo de agua y vida, siendo uno de los departamentos con mayor presencia de fuentes hídricas dulces de Colombia, que en conjunto con las densas selvas que se levantan sobre su territorio, la convierte en el hogar de miles de especies de fauna y flora, que aporta una gran proporción del oxígeno para un país, que cada día se ahoga en el efecto invernadero.', 'Este martes 27 de diciembre, cuatro días después de que una embarcación quedara encallada en el río Putumayo y causara derrame de combustible, las autoridades de socorro del departamento recomendaron a la población aledaña no consumir el agua ni el pescado de la fuente hídrica. Además, están buscando alternativas para resolver esta crisis ambiental mediante un puesto de mando unificado (PMU).', 'En Leticia, capital del Amazonas, hay otro grave problema ambiental: sus humedales están seriamente amenazados por, entre otras cosas, residuos sólidos y cultivo de peces no nativos. Esta situación también está causando una difícil situación de salud pública.', 'Por la destrucción de las dragas que son utilizadas para la extracción minera en el rio puré y cotuhé de las amazonas, se genera un gran impacto ambiental, ecosistemas y afluentes hídricos, en el marco de estas operaciones se estarían vertiendo alrededor de entre 5 y 7 gramos de mercurio por cada gramo de oro que se estarían extrayendo de los afluentes hídricos.', 'El estudio Contenido de mercurio en comunidades étnicas de la subregión planicie de la Amazonia colombiana arroja datos alarmantes sobre las consecuencias del mercurio en la salud humana y en el ecosistema de los ríos Caquetá, Apaporis, Puré y Cotuhé. Este territorio ha estado expuesto a varias olas extractivas y conflictos que ponen en riesgo sus ecosistemas. Uno de los fenómenos que ha afectado la zona es la minería ilegal, actividad en la que se utiliza el mercurio, dice el estudio realizado a través de la Dirección Territorial Amazonia, y con el apoyo de la Universidad de Cartagena y la Fundación Gordon and Betty Moore.', 'la alcaldía de San Vicente del Caguán dio a conocer que “debido a los últimos acontecimientos ocurrido en las plataformas Capella, en la vereda los Pozos y Bruselas, se han ocasionado derrames de sustancias químicas que pueden llegar a contaminar fuentes hídricas cercanas”.', 'El mercurio dorado amenaza el suelo y las aguas de la AmazoniaA la Amazonia se le reconoce como el bosque tropical y el sistema de ríos más grande del mundo. Es el lugar que alberga el 10% de la biodiversidad del planeta y se extiende por los territorios de 9 países en Latinoamérica. Es el hogar de más de 47 millones de personas, incluyendo 410 grupos indígenas.', 'Incendios y derrames de hidrocarburos amenazan la Amazonía colombianaLos incendios masivos en la Amazonía colombiana superan las cifras para el mes de enero de los últimos diez años, a lo que se añade al menos dos derrames de hidrocarburos.', 'El 40 % del Amazonas puede convertirse en un ecosistema tipo sabana.Una gran parte de la selva amazónica, un 40 %, está en riesgo de convertirse en un ecosistema del tipo sabana, mientras aumenta el calentamiento de la región por las emisiones contaminantes', ' El Amazonas podría haber llegado a un punto de no retorno: la deforestación y los fuegos han llevado al extremo a la mayor selva tropical del mundo las tres primeras semanas de agosto 71.497 focos de incendio, el mayor número para el período en los últimos siete años, y poco más de la mitad de los cuales se reportaron en la Amazonía', 'reportaron más de 1.200 derrames de petróleo en el área. Las poblaciones indígenas, así como la flora y fauna de la selva, sufren diariamente las consecuencias de manera más directa. la situación en el Amazonas empeora debido a las fallas de las autoridades para tratar con los perpetradores y prevenir los derrames', 'Contaminacion',
        'La contaminación en el Amazonas es un problema creciente que pone en peligro la biodiversidad y las comunidades que dependen de esta selva. La tala ilegal, la minería descontrolada y los vertidos tóxicos están causando un daño irreparable.', 'La deforestación en el Amazonas está liberando grandes cantidades de carbono a la atmósfera, contribuyendo al cambio climático global y afectando el equilibrio ecológico de la región.', 'La quema de bosques en el Amazonas para crear áreas de pastoreo y cultivos libera humo y gases contaminantes, provocando problemas respiratorios y afectando la calidad del aire en la región.', 'La preservación del Amazonas es urgente para evitar la pérdida de biodiversidad, proteger las comunidades indígenas y mitigar los impactos del cambio climático. Es necesario tomar medidas para reducir la contaminación y promover prácticas sostenibles en la región.',
        'mineria', 'La minería ilegal de oro se expande sin freno en la Amazonía,como demuestran los últimos datos en la provincia de Napo, en Ecuador, la multiplicación de actividades mineras ilegales altamente contaminantes es imparable en la Amazonía', 'La minería ilegal crece voraz y amenazante en la Amazonía,El Gobierno, en cambio, lo atribuye al poder económico del narcotráfico y, si bien existen focos en casi todo el país, hay cinco en la Amazonía que generan preocupación por su extensión o por estar dentro de reservas naturales.', 'La minería ilegal de oro ha sido una importante fuente de ingresos para los grupos armados no estatales en Colombia desde finales de la década de 1990, cuando el Bloque Central Bolívar (BCB) de las Autodefensas Unidas de Colombia (AUC), una organización paramilitar de derecha comenzó a beneficiarse de la extracción del mineral en los departamentos de Bolívar y Antioquia.', 'Más de 30 hectáreas afectadas, la desaparición de una quebrada y la contaminación de varias fuentes hídricas es el costo medioambiental que deja la explotación de este yacimiento de oro a cielo abierto. Nueve personas fueron detenidas, Su nocivo impacto ambiental, por tanto, tampoco da tregua. Un ejemplo de su implacable efecto sobre el medioambiente fue el que encontraron en las últimas horas tropas del Batallón de Artillería Nº. 27, de la Vigésima Séptima Brigada del Ejército Nacional, en la vereda conocida como Las Minas, del corregimiento de Puerto Umbría, en el municipio de Villagarzón (Putumayo), Allí no solo resultaron afectadas cerca de 30 hectáreas de vegetación nativa por cuenta de una mina de oro a cielo abierto, sino además una quebrada, la cual -según las autoridades- prácticamente desapareció. ', 'La minería en los ríos de esta zona no es ilegal solo porque carezca de permisos y títulos. También lo es porque viola las leyes de origen de los pueblos indígenas amazónicos como los bora miraña, makuna, uitoto y aduche, a lo largo de las cuencas. Según su cosmogonía, son los espíritus guardianes del agua los únicos que pueden autorizar la extracción del oro. A ellos nadie les ha vuelto a preguntar “y por eso es que llegan las desgracias”, afirma la lideresa uitoto Nazareth Cabrera.“Los dineros de la minería salen de los lugares en donde se explota. Lo que sí quedan son las consecuencias ambientales y sociales”,  explica María Camila Munar, asesora de la Fundación GAIA Amazonas.', 'Sin el servicio de acueducto se encuentran los municipios de Curillo, Solita y Solano, en el sur del Caquetá, luego de que, a través de redes sociales, circularan imágenes en las que se envidenciaban una mancha de petróleo que cayó al río Caquetá, frente a esta situación, el director territorial de Corpoamazonia, Mario Ángel Barón, manifestó que la emergencia ambiental se habría generado por la perforación de una línea de conducción de petróleo, en la zona baja del vecino departamento del Putumayo.', 'Sin importar la profundidad y la corriente del río Guaviare, mineros de subsistencia arriesgan su vida a diario en las aguas, sin que ninguna autoridad legal medie para proteger este sector de la economía informal, un sin numero de personas llegan a Puerto Tolima desde muy temprano a recolectar gravilla desde la profundidad del caudaloso río con un balde con el cual se introducen hasta el fondo para pescar el material y arrojarlo en una canoa, por cada metro cubico reciben 10.000 pesos y en el día pueden llegar a conseguir unos 50.000 pesos para alimentar su familia.', 'Desde que comenzó la pandemia la minería ilegal en la Amazonia colombiana ha incrementado. Además de las problemáticas ambientales que trae el mercurio utilizado en estas actividades, monitoreos satelitales evidenciaron que desde 2020 ha aumentado más del 1.000 % sobre el río Puré y los mineros tienen enclaves en el Parque Nacional Natural Río Puré, a pocos kilómetros de un pueblo en aislamiento.', 'Según monitoreo de la Alianza Regional Amazónica para la Reducción de los Impactos de la Minería de Oro, integrada por varias organizaciones de la sociedad civil, se ha identificado un aumento de lanchas, balsas y dragones en ríos como Puré, Cotuhé y Caquetá, en donde la explotación de oro puede estar realizándose tanto por mineros colombianos como brasileños. En el río Puré (ubicado en el departamento del Amazonas, entre los ríos Caquetá y Putumayo) durante 2022 se identificaron 357 balsas y dragones, un incremento de más del 1.000 % con respecto a 2020, cuando se reportaron 25. Estos registros se llevan a cabo, entre otros recursos, gracias a imágenes satelitales. ', 'Desde que comenzó la pandemia la minería ilegal en la Amazonia colombiana ha incrementado. Además de las problemáticas ambientales que trae el mercurio utilizado en estas actividades, monitoreos satelitales evidenciaron que desde 2020 ha aumentado más del 1.000 % sobre el río Puré y los mineros tienen enclaves en el Parque Nacional Natural Río Puré, a pocos kilómetros de un pueblo en aislamiento.', 'Una operación especial de las Fuerzas Militares y la Policía en el marco del Plan Perseo ha destruido los equipos de producción minera que, según las investigaciones, pertenecían a grupos armados vinculados al narcotráfico.', 'la corte constitucional ordenó suspednder cualquier actividad minera en un millon de hectareas en los departamentos  de vaupés y Amazonas.', 'En junio de 2021 varias comunidades indígenas vieron cinco embarcaciones haciendo minería ilegal en el río Caquetá. Imágenes satelitales llegaron a mostrar hasta 19 embarcaciones en el mismo mes en el Río Puré, uno de sus afluentes.', 'En lo que va de 2022 y hasta finales de mayo, en el área central del departamento del Chocó, la Fuerza Pública junto con la Fiscalía General de la Nación y la Corporación Autónoma Ambiental del Chocó, Codechocó, ubicaron y destruyeron 20 maquinarias amarillas y 11 dragas y dragones. También destruyeron 49 unidades de producción minera ilegales, 27 motores, y un bote.', 'La extracción ilícita de minerales no para en diferentes regiones del país, pese a las restricciones existentes de movilidad para controlar la pandemia del coronavirus.                                                                                                                Su nocivo impacto ambiental, por tanto, tampoco da tregua. Un ejemplo de su implacable efecto sobre el medioambiente fue el que encontraron en las últimas horas tropas del Batallón de Artillería Nº. 27, de la Vigésima Séptima Brigada del Ejército Nacional, en la vereda conocida como Las Minas, del corregimiento de Puerto Umbría, en el municipio de Villagarzón (Putumayo). ', 'La minería ilegal en la Amazonía colombianaA medida que los precios del oro se han disparado, los grupos criminales que antes se dedicaban exclusivamente al tráfico de drogas y armas han incursionado en la minería ilegal.', 'Cultivos de coca y minería ilegal se dispararon en la Amazonía, advierte organización ambientalistaEste jueves el World Wildlife Fund (WWF) dio a conocer el informe ‘Un clima peligroso: deforestación, cambio climático y violencia contra defensores ambientales en la Amazonía colombiana’ en el que señalan que la desmovilización y el desarme de la Farc creó una especie de vacío de poder en esa zona lo que aumentó la explotación de recursos y de actividades económicas ilegales.', 'Cultivos de coca y minería ilegal se dispararon en la Amazonía, advierte organización ambientalistaEste jueves el World Wildlife Fund (WWF) dio a conocer el informe ‘Un clima peligroso: deforestación, cambio climático y violencia contra defensores ambientales en la Amazonía colombiana’ en el que señalan que la desmovilización y el desarme de la Farc creó una especie de vacío de poder en esa zona lo que aumentó la explotación de recursos y de actividades económicas ilegales.', 'Destrucción del Amazonas: autoridades colombianas detuvieron brasileros que estaban afectando ocho kilómetros de río y selvaLos individuos llevaban a cabo actividades de minería ilegal en la zona en complicidad con el grupo armado organizado residual Estructura 48, Comandos de Frontera',  'mineria', 'Las carreteras, las vías férreas y las nuevas rutas de transporte fluvial están transformando la Amazonia. Actualmente decenas de miles de kilómetros de carreteras atraviesan sus bosques; y la construcción de nuevas rutas seguirá aumentando. En este momento más de 20 proyectos de construcción de carreteras gigantes ejercen presión sobre la Amazonia, lo que puede generar un rápido aumento de la pérdida de bosques.', 'En los últimos tres meses de 2019, la deforestación en Colombia se concentró en un 84.9% en la Amazonia, de acuerdo con el Instituto de Hidrología, Meteorología y Estudios Ambientales (Ideam).', 'Colombia propone fondo internacional para proteger la Amazonía. El presidente de Colombia, el izquierdista Gustavo Petro, anunció el jueves (18.08.2022) que pedirá a los países ricos y a grandes empresas pagar a los campesinos por cuidar la selva amazónica y recuperar las zonas deforestadas.', '"La deforestación en Colombia subió 11 % en el primer semestre de 2022. En las últimas semanas, se ha tendido en medio de la opinión pública las cifras sobre deforestación en Colombia presentadas por el Ministerio de Ambiente y Desarrollo Sostenible, después de analizar los reportes del Sistema de Monitoreo de Bosques y Carbono del Instituto de Hidrología, Meteorología y Estudios Ambientales (Ideam). De acuerdo con la cartera, en las últimas dos décadas se han deforestado en el país 3,1 millones de hectáreas de bosque, siendo la Amazonia colombiana, el sector más afectado.', 'La crisis con las disidencias de Iván Mordisco impacta la lucha contra la deforestación', '"El ""impacto inmenso"" de las regiones del Amazonas que ahora emiten más carbono del que absorben. vLa deforestación y el cambio climático están alterando la capacidad de la selva amazónica de absorber dióxido carbono, de acuerdo con un nuevo estudio.Significativas áreas de la selva tropical más grande del planet han comenzado a emitir más CO2 del que absorben.El área sureste es la más afectada, dicen los científicos, con tasas más altas de pérdida de árboles y un incremento en el número de incendios.Las temperaturas allí han aumentado tres veces el promedio mundial durante los meses más calurosos."',
        '"El Amazonas está cerca del punto de inflexión de convertirse en una sabana, sugiere un estudio. La selva del Amazonas podría estar acercándose a un punto de inflexión crítico que podría hacer que este ecosistema biológicamente rico y diverso se transforme en una sabana de hierba.El destino de la selva tropical es crucial para la salud del planeta, ya que alberga una variedad única de vida animal y vegetal, almacena una enorme cantidad de carbono e influye en gran medida en los patrones climáticos globales."', 'McDonald’s Corp. estaría vinculada con la deforestación y abusos laborales en la selva amazónica,hay varios casos en los que la carne de res de ranchos despejados ilegalmente se mezcló con la de granjas para ocultar su verdadero origen y luego se envió a mataderos propiedad de empresas que abastecen a McDonald’s.', 'Siete de cada diez habitantes de la Amazonía no tienen acceso al agua potable, pese a albergar el 20% de las reservas de agua dulce del planeta, según World Vision que ha lanzado la campaña Iniciativa de la Cuenca Amazónica, coincidiendo con el Día Mundial del Medio Ambiente.', 'A la Amazonia se le reconoce como el bosque tropical y el sistema de ríos más grande del mundo. Es el lugar que alberga el 10% de la biodiversidad del planeta y se extiende por los territorios de 9 países en Latinoamérica. Es el hogar de más de 47 millones de personas, incluyendo 410 grupos indígenas. Por su extensión y características ambientales, esta región es fundamental para la regulación del clima, el ciclo hídrico y para la prosperidad ambiental y económica de la zona. Por ello, su protección y conservación es vital para el ecosistema mundial.', 'Durante agosto del 2019, el aumento de los incendios en Brasil y Bolivia volvió a poner el Amazonas en el punto de mira.Según un informe del Instituto de Investigación Ambiental de la Amazonía (IPAM) los fuegos estaban directamente relacionados con la deforestación.A pesar de las promesas del gobierno de que pondría fin a la deforestación, los incendios en el bosque volvieron a ocurrir durante el mismo periodo de 2020.', 'La tala ilegal, la ganadería y la expansión de tierras agrícolas han destruido un 75 % de los bosques del departamento en las últimas décadas, la deforestación y el conflicto han sido dos de los problemas más significativos que han afectado durante décadas al departamento de Caquetá, Como otras regiones del sur del país, Caquetá ha sido sitio de violencia intensa y degradación ambiental, con ambos problemas que, a menudo, se interceptan y exacerban el uno al otro, La deforestación es uno de las dificultades ambientales más importantes en Colombia, ya que el país pierde un estimado de 220.000 hectáreas de bosque al año, pero en Caquetá, la deforestación ha sido particularmente grave.', 'A pesar de que la deforestación está prohibida en el Guaviare por el Estado colombiano y las disidencias, la selva amazónica, que ocupa prácticamente la totalidad del territorio, sigue menguando año a año, agonizando entre llamas, y dando paso a un inmenso potrero.Cuesta pensar en que en algún momento todo estuviera lleno de selva, como cuentan los habitantes del Guaviare, pues los parches “tumbados”, cuadrados perfectos de bosque quemado y deforestado, se pueden apreciar desde el aire pero, sobre todo, transitando las “trochas” (caminos) que conectan municipios y veredas en las que el ojo no alcanza a ver dónde acaban los potreros de cientos de hectáreas en los que contrasta la poca cantidad de vacas.', 'La Amazonia colombiana concentró más del 57% de las alertas tempranas por deforestación que acaban de ser publicadas por el Ideam. La Macarena, Sierra Nevada de Santa Marta y Chiribiquete fueron los Parques Nacionales más afectados por la motosierra, En el primer trimestre, este departamento perdió más de 14.000 hectáreas boscosas y concentró 24,3 por ciento de las alertas tempranas por deforestación, cifra que en el mismo periodo de 2018 fue del 12,1 por ciento. El Ideam identificó 305 parches con áreas superiores a las 10 hectáreas, de los cuales 13 superaron las 50 hectáreas. ', 'Líderes indígenas reunidos este martes en Lima (Perú) advirtieron que la Amazonia se encuentra en “alerta roja”, porque el 26 % de su ecosistema ha sido destruido irreversiblemente por la deforestación, la contaminación y el narcotráfico.', 'Cuando alguien se atrevía a afirmar o a vaticinar que la selva amazónica sería tan solo un buen recuerdo si la deforestación y los incendios se seguían produciendo en ella, muchos se iban en contra de quien pronunciaba dichas palabras y lo consideraban exagerado, pues resulta que al paso que vamos, esos comentarios parecerán una dura sentencia contra una región que no para de arder.', 'Eso es lo que está pasando en San Vicente del Caguán, el municipio más deforestado del Caquetá con 18.000 hectáreas derribadas el año pasado, desde principios de agosto varios pobladores de la zona han ingresado a un área de reserva forestal instituida por la propia comunidad para tumbar el bosque y establecer pastos para la ganadería. El saldo, dice él, son al menos 2.000 hectáreas deforestadas hasta ahora y con grandes posibilidades de aumentar en los próximos meses.', 'La mayor parte de la deforestación se debe a la construcción de vías terciarias, no porque destruyan los bosques, sino porque es por ahí que llegan los nuevos colonos. Le sigue la tala y quema para ganadería, y en tercer lugar, los cultivos de coca que han aumentado en el departamento. Esto ocurre en zonas con bosques que tardaron miles de años en crecer y que son el hogar de una flora y fauna única.', 'Los municipios de Puerto Guzmán, Puerto Leguízamo, Puerto Asís y Puerto Caicedo registraron cerca de 11 mil hectáreas deforestadas, lo que corresponde al 85 % de zonas afectadas por esta problemática ambiental en el Putumayo.                                                                 Los municipios con mayor deforestación han sido Puerto Guzmán, con alrededor de 1.953 hectáreas, y Puerto Leguízamo, con 1.144 hectáreas, en el 2021',  '“A pesar de que ya hay sitios con pocos remanentes de bosques, el tamaño de la deforestación y de las quemas sigue siendo enorme. Hay sitios que están quedando con un cambio de cobertura definitivo, algunos de ellos corresponden a corredores de conectividad ecológica que eran cruciales”, explica Botero, ya en tierra, en la sede de la FCDS en San José del Guaviare, a manera de balance. “Estamos entrando en un punto de inflexión. Es posible que antes haya habido más área deforestada, pero no necesariamente con esta intensidad”.', 'La Amazonía colombiana sufre una "preocupante degradación ambiental"Cultivos ilícitos, agricultura intensiva, minería ilegal narcotráfico producen una "preocupante degradación ambiental" en la Amazonía colombiana, cada vez más golpeada por la deforestación y los ataques contra ambientalistas, denunciaron este jueves varias ONGs.',
        'El ganado acorrala a la Amazonía Desde 2016 la población vacuna que circunda a los parques nacionales colombianos La Macarena, Cordillera los Picachos, Tinigua y Serranía del Chiribiquete se ha más que duplicado. Quebrantan así la ecología de reserva que protege la selva más importante del planeta. ¿Por qué y cómo se han podido apropiar los particulares de tierras públicas reservadas?', 'Arde la Amazonía colombianaLas llamas devoran, una vez más, la Amazonía colombiana. Pocos minutos después de que la avioneta despega del aeropuerto de Villavicencio, la puerta de entrada a la inmensidad de las llanuras que dominan la mitad sur de Colombia, se comienza a observar las copiosas columnas de humo. ', 'La crisis con las disidencias de Iván Mordisco impacta la lucha contra la deforestación', 'EL PESO DE LA COCA EN LA DEFORESTACION AMAZONICA El cultivo de la hoja de coca en la región de la Amazonia viene desde los tiempos ancestrales. Las comunidades amazónicas cuentan que los grupos indígenas que han habitado este territorio históricamente han sembrado coca, entre ellos los huitotos, muinane, andoques, nonuyas, mirañas, yucunas y matapís.', '“Ganadería genera 60 veces más deforestación que el área por coca en la Amazonía”Así lo concluye Paulo Murillo, topógrafo y PhD en geografía de la Oregon State University, en su tesis doctoral sobre la expansión de los cultivos de coca y ganadería extensiva en el posconflicto. Afirma que habitantes de los territorios se debaten entre su supervivencia y la protección del medioambiente.', 'Ganadería extensiva, uno de los motores de la deforestación en ColombiaSegún cifras del Ideam, solo en 2021 se deforestaron 174.113 hectáreas en el país, especialmente en la Amazonía.', 'SOS por la Amazonía colombiana: 600.000 hectáreas han sido arrasadas en los últimos 5 años',
        'El aumento de las represas en la Amazonia amenaza el flujo natural de sus ríos, altera los ciclos naturales y pone en grave riesgo especies como los delfines y peces migratorios. El suministro de agua para las comunidades locales y el transporte de alimentos, también se ven afectados por cuenta de la producción de energía en la selva amazónica. En la Amazonia hay 154 represas para la producción de energía hidroeléctrica y se planea la construcción de otras 277 en los próximos años.', 'Desde hace dos semanas el nivel promedio del río Amazonas, en el lado brasileño, ha llegado a bajar 25 centímetros por día,Ríos reducidos a tierras agrietadas y pequeñas embarcaciones atascadas e inmovilizadas ante la ausencia de agua, se estan afectando 41 municipios que están en alerta, 3 en estado de emergencia, Y los 18 restantes tienen problemas de abastecimiento y salida de producción.', 'los problemas del eje vial que une a 12 distritos que siembran más de 40 mil hectáreas de café y producen más de un millón de quintales de este preciado grano para exportación, la reclasificación de una ruta nacional definitiva; la construcción de un pavimento de tránsito pesado y la conectividad digital en la zona pues carece de este importante servicio en muchas localidades.', 'La anomalía de la temperatura del agua del océano Pacífico ha mostrado un calentamiento bastante atípico durante algunas semanas que podría provocar efectos más intensos y duraderos, como una sequía en el norte de Brasil, dejando a la Amazonia bajo un gran estrés hídrico y más propensa a los incendios.', ' El delfín rosado ha sido cazado por que la carne del animal es usada como carnada para pescar el pez mota, Esta actividad no solamente es ilegal sino que estan cazando delfines de forma cruel.', ' las carreteras que actualmente están proyectadas para ser construidas en la región amazónica producirán desarrollo, también provocarán deforestación, violaciones a derechos indígenas, transgresiones a las normas ambientales y pérdidas económicas. ', 'Así operan los grupos armados y el narcotráfico en el Amazonas. De alguna manera esta industria ha sido la vía del lavado de dinero de narcos desde que aparece este ilícito. Pero hasta los noventa se fortalece y se hace visible esta acción con los herederos de los carteles de la droga, quedando a luz pública, por sus estrategias que quedaron en evidencia ante el mundo por sus excedidas opulencias y los seguimientos que desenmascararon terceras personas como a empresas de fachada.', 'Interceptan aeronave que intentaba traficar droga y detienen al piloto y copiloto, “Estos aviones vienen desde Suramérica y aquí en Venezuela son interceptados con nuestro Sistema de Defensa Aeroespacial Integral, capaz de detectar todo tipo de aeronaves que ingresen; inmediatamente procedemos a los protocolos de actuación de acuerdo a los estándares internacionales”, aseguró Ichaso quien destacó que Venezuela ejerce su soberanía sobre la normativa de los espacios aéreos', 'en el operativo se encontraron 12 computadores portátiles –que tendrían claves sobre las finanzas de la red ilegal– y una nevera en la que se guardaban licores finos para el consumo de ‘Gentil Duarte’, en el cambuche principal estaba la cama del jefe de la disidencia y había ropa de mujer, Además se encontraron 60 equipos de campaña nuevos, 200 kilos de víveres, 9 fusiles, 8 pistolas nueve milímetros, miras telescópicas y 2.547 cartuchos de diferentes calibres, la mayor parte del campamento sigue en pie y sin daño, pues el bombardeo se concentró solo en el sitio que se sabía estaba destinado para la llegada de ‘Gentil Duarte’, quien pasaría la noche en e l lugar.', 'El asesinato de cuatro adolescentes indígenas en el sur de Colombia supone un nuevo obstáculo para el objetivo de "paz total" que persigue el gobierno de Gustavo Petro, los jóvenes fueron reclutados forzosamente por las disidencias de la guerrilla de las FARC en su propio territorio y luego, a pesar de que ellos mismos lograron escapar, acabaron perdiendo la vida, la masacre en la que fueron asesinados ocurrió el pasado 17 de mayo en el departamento del Putumayo. El hecho fue denunciado tres días después por la Organización Nacional de los Pueblos Indígenas de la Amazonía Colombiana (Opiac) en redes sociales.', 'Un operativo en conjunto con la Policía Nacional, el Ejército y la Fiscalía permitió darle un golpe al grupo armado  ‘Comandos de Frontera’ de la llamada ‘Segunda Marquetalia’, la operación que se hizo en el departamento de Caquetá logró la captura de siete integrantes del GAOR estructura 48 de esta disidencia, Los capturados deberán responder por delitos como concierto para delinquir agravado, homicidio agravado y tráfico, fabricación o porte de armas de fuego.', 'La Fuerza Pública de Colombia no descansa en la lucha contra el narcotráfico, logrando resultados cada vez más contundentes. A finales de octubre las autoridades destruyeron 71 laboratorios ilegales de organizaciones narcotraficantes en la región Amazónica colombiana, informó la Armada de Colombia en un comunicado.', 'Los disidentes de las Farc preguntaban por la ubicación de 450 kilos de marihuana de variedad cripa, que estaban ocultos en algún lugar del vasto Amazonas. El traficante se negaba a entregar el botín y soportó durante ocho horas los más crueles castigos.', 'Las conclusiones, recopiladas en un artículo recientemente publicado en la revista Nature, surgen al comparar la capacidad de resistir la falta de agua de las diferentes especies de árboles de la región y constatar que la tolerancia a la sequía de los árboles de la selva amazónica puede variar en función de la especie.', 'MÁS de un tercio de la Amazonía se habría degradado por la actividad humana y las sequías, dijeron investigadores, y se requieren acciones para proteger ese ecosistema crucial.', 'Uno de los panoramas que evidenció el Sinchi (Instituto Amazónico de Investigaciones Científicas) tiene que ver con el cambio climático donde se indicó que, si se pierde el 40% de la selva tropical, los bosques podrían dirigirse a un punto de inflexión o punto de no retorno, donde el bosque restante eventualmente transformarse en un ecosistema de sabana. En Colombia la región amazónica representa el 42,3% del territorio con una superficie estimada de 483.164 km. En esta área, el 14% está dominado por tierras agrícolas, vegetación secundaria y bosques fragmentados. Actualmente, el 86% del área corresponde a áreas naturales en buen estado de conservación, donde los bosques son la cobertura dominante.', ' Ningún continente se ha librado del fuego, pero la zona de la que más se habla es de la gran cantidad de incendios que están destruyendo el Amazonas. Aún no se puede saber si el área afectada por incendios es mayor que la del año pasado, pero sí se conoce que, como apuntan desde Greenpeace, el número de incendios surgidos entre enero y finales de agosto ha sido un 145% mayor que en el mismo período de 2018. Aunque también es cierto que el número de incendios de este año no es mayor que la media de los últimos 18 años (desde que los satélites de la NASA son capaces de monitorizarlos), según los datos científicos.', 'Las poblaciones del Amazonas sufrieron una selección natural a causa del patógeno del Chagas y su genética se fue adaptando hasta tener una variante resistente a esta enfermedad, según ha descubierto un estudio coliderado por el Instituto de Biología Evolutiva (IBE-CSIC-UPF) y la Universidad de Sao Paulo (Brasil).', 'La Fuerza Pública de Colombia no descansa en la lucha contra el narcotráfico, logrando resultados cada vez más contundentes. A finales de octubre las autoridades destruyeron 71 laboratorios ilegales de organizaciones narcotraficantes en la región Amazónica colombiana, informó la Armada de Colombia en un comunicado.', 'En Güerima, Vichada, a las puertas de la Amazonía colombiana, las antiguas pistas del narcotráfico sirven para transportar cacao a los campesinos que abandonaron los cultivos ilícitos en esta región pobre, despoblada y sin carreteras', 'Sobre la ciudad de San José del Guaviare, un remanso en la Amazonia colombiana, un helicóptero de la época de la guerra en Vietnam repleto de agentes de la policía judicial y el Ejército se eleva hacia el cielo. A medida que vuela hacia el sur, el mosaico de fincas ganaderas atravesado por caminos de arcilla roja se disuelve en parches de vegetación desfigurados por la deforestación. ', 'Tras una constante presión militar ejercida por tropas del Ejército Nacional y de la Armada de Colombia, que inició en el departamento de Vichada, fue incautada más de media tonelada de clorhidrato de cocaína, que era transportada a bordo de una embarcación sobre el río Guainía, en el municipio de Inírida.', 'La búsqueda por encontrar una gota de agua es algo que a los habitantes del departamento del Caquetá cada vez se les hace más difícil. El agua potable en esta parte del país está ausente y niños, jóvenes, adultos e indígenas tratan de hallar alguna solución.',
        'Los acuerdos se dieron en una reunión entre gobernadores indígenas de diferentes etnias, líderes de Acilapp y funcionarios del Instituto Colombiano de Desarrollo Rural (Incoder), que los avalaron.                                                         Más pastos, menos selva. La invasión de tierras también la sufren siete familias de la etnia coreguaje, que pertenecen al asentamiento Chaibajú.', 'Según la OPIAC, los menores fueron asesinados entre el Amazonas y el Caquetá, al sur de Colombia.                    La matanza de 4 menores en colombia por la que el presidente petro suspendio el cese al fuego con las disidencias de las FARC . El hecho fue denunciado tres días despues por la organizacion Nacional Nacional de los pueblos indígenas de la Amazonía Colombia (OPIAC) en redes sociales. ', 'El bajo nivel del río afecta ya a todos los 62 municipios que componen el estado brasileño de Amazonia. En algunos puntos en los que el río todavía fluye, el nivel de profundidad es de hasta 35 centímetros, por lo que las imágenes de pescadores arrastrando sus embarcaciones se torna bastante común. En la fotografía, un barco varado por las sequías en Lagoa da Francesa, en el municipio de Parintinis, el 21 de octubre de 2022.', 'Este ecosistema podría perder su capacidad de absorción de carbono, ha comenzado a emitir más CO2 del que absorbe debido a la creciente deforestación y los incendios, se debe principalmente a la actividad humana relacionada con la tala que ha enfrentado en los últimos año, al menos tres cuartos de la Amazonía han perdido la capacidad de recuperarse de fenómenos adverso.', 'El Igac dice que los suelos del Caquetá y Putumayo son los más afectados en la Amazonia colombiana. La degradación de este recurso natural por la pérdida de bosque en las zonas montañosas intensifica los derrumbes y desbordamientos de los ríos.', 'Por un período de diez años, los investiga+A2:A32 ayudar a combatir el cambio climático con su capacidad de absorber carbono de la atmósfera; y por otro, nos ayudará a entender cómo la selva se verá impactada por esos cambios", dijo David Lapola, investigador de la Universidad de Campinas, que coordina el proyecto junto a Quesada.', 'IOHFVJSBD SJDFC SDFDCH DU UHSDF USDNS USB SJSF USF UWF USWOEQYEOW EYERP AUSNC QIANZCO DJSFHUWE QGRBKXX ANSAS IXBUABXJ SUZDBS', "                                    "
    ]

    etiquetas = ['CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'CONTAMINACION', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'MINERIA', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION',
                 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'DEFORESTACION', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO', 'NINGUNO']
    # Agregamos una lista de textos y sus correspondientes etiquetas de categoría obtenidas de la base de datos dada.

    # 2. Extracción de características
    vectorizador = CountVectorizer()
    caracteristicas = vectorizador.fit_transform(textos)
    # Se crea un objeto CountVectorizer, que se utiliza para convertir los textos en características numéricas.
    # La función fit_transform() ajusta el vectorizador a los datos de entrenamiento dados y luego transforma estos textos en una representación numérica.

    # 3. Entrenamiento del modelo
    modelo = MultinomialNB()
    modelo.fit(caracteristicas, etiquetas)
    # Se crea un objeto de clasificador de Naive Bayes multinomial (MultinomialNB)
    # El objeto se entrena con las características y etiquetas de los datos dados anteriormente

    nuevos_textos = [text]

    # 5. Predicción de la categoría:
    nuevas_caracteristicas = vectorizador.transform(nuevos_textos)
    # Utilizamos vectorizador (previamente ajustado), esto transforma los nuevos textos en características numéricas.
    # La función transform() convierte los textos en la representación numérica utilizada durante el entrenamiento del modelo.

    # 6. Predicción de las categorías
    predicciones = modelo.predict(nuevas_caracteristicas)
    # Utilizando el modelo entrenado modelo, se realizan predicciones sobre las nuevas características.
    # El método predict() del modelo devuelve la categoria predichas para el texto dado.

    # 7. Resultados
    for texto, categoria in zip(nuevos_textos, predicciones):
        # Se utiliza for en combinación con la función zip() para iterar simultáneamente sobre el texto y la prediccion.
        if categoria not in etiquetas:
            categoria = "Ninguno"
        # se verifica si la categoría predicha se encuentra en la lista de etiquetas. Si no, se asigna la categoría como "Ninguno".
        return categoria
        # se devuelve la categoría predicha

Extracción de fechas con base al "string" enviado por el usuario

In [ ]:
def get_dates(text: str) -> list:
    return re.findall('(19[89][0-9]|20[0-4][0-9]|2050)', text) + re.findall('([a-zA-Z]+) del (\d{4})', text)


La siguiente función crea un diccionario con el texto y listas vacías para las entidades nombradas, las fechas y el impacto. Llama a la función get_impact para obtener la categoría de impacto, fechas, identifica y claisifa entidades según el tipo (persona, lugar, organización, u otro) del texto y la guarda en el diccionario. Posteriormente, se abre el archivo de salida en modo escritura y guarda el diccionario en formato JSON.


In [ ]:
def ner_from_str(text: str, output_path: str) -> None:
    d = {"text": text,
         "org": [],
         "loc": [],
         "per": [],
         "dates": [],
         "misc": [],
         "impact": ""
         }
    d["impact"] = get_impact(text)
    d["dates"] = get_dates(text)
    sentence = Sentence(text)
    TAGGER.predict(sentence)
    for entity in sentence.get_spans('ner'):
        ent, clss, conf = entity_from_tagged_str(entity)
        if clss == 'PER':
            d["per"].append(ent)
        elif clss == 'LOC':
            d["loc"].append(ent)
        elif clss == 'ORG':
            d["org"].append(ent)
        elif clss == 'MISC':
            d["misc"].append(ent)
    with open(output_path, "w") as outfile:
        json.dump(d, outfile)

Esta función lee un archivo de texto dado su ruta y devuelve su contenido como una cadena. Si el archivo no existe o hay algún otro error, imprime un mensaje de error y devuelve un mensaje de alerta.

In [ ]:
def obtener_contenido_archivo(archivo_path: str) -> str:
    try:
        with open(archivo_path, "r") as file:
            contenido = file.read()
        return contenido
    except FileNotFoundError:
        print("Ingrese un archivo", archivo_path)
        return None
    except Exception as e:
        print("Ingrese un archivo", e)
        return None

In [ ]:
def ner_from_file(text_path: str, output_path: str) -> None:
    text = obtener_contenido_archivo(text_path)
    ner_from_str(text, output_path)

Esta función obtiene el texto de una página web dada su url, usando la librería requests para hacer la petición y la librería BeautifulSoup para analizar el contenido HTML.


In [ ]:
def getdata(url: str) -> str:
    r = requests.get(url)
    htmldata = r.text
    soup = BeautifulSoup(htmldata, 'html.parser')
    data = ''
    text = ''
    for data in soup.find_all("p"):
        text += data.get_text()
    return text

In [ ]:
def ner_from_url(url: str, output_path: str) -> None:
    text = getdata(url)
    ner_from_str(text, output_path)

#Funciones para procesar, categorizar y etiquetar noticias de diversas fuentes



In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
ner_from_str("Bogotá D.C., 16 de mayo de 2023 (@Minambienteco) – La ministra de Ambiente y Desarrollo Sostenible, Susana Muhamad, reveló que la deforestación en la Amazonía colombiana cayó un 25% en el año 2022 en comparación con el año 2021, según las cifras preliminares de deforestación reportadas por el Sistema de Monitoreo de Bosques y Carbono del Instituto de Hidrología, Meteorología y Estudios Ambientales (Ideam).", "salida.json") #-> None

In [ ]:
ner_from_file("/content/noticias_formatocsv.csv", "salida_noticias_depuradas.json")

In [ ]:
ner_from_url("https://elpais.com/america-colombia/2023-06-09/hallados-con-vida-los-cuatro-ninos-perdidos-hace-40-dias-en-la-selva-colombiana.html", "salida_de_url.json")

##👩‍💻🌎📝📜😊

